# Reference
+ [pkl文件与pickle.dump，pickle.load](https://blog.csdn.net/weixin_43135178/article/details/116430144)

In [1]:
import fastai
print("fastai:", fastai.__version__)

from platform import python_version
print("python_version:", python_version())

import torch
print("torch version:", torch.__version__)

fastai: 2.7.10
python_version: 3.7.9


OSError: [WinError 193] %1 is not a valid Win32 application

## Install Package

In [4]:
import fastbook
fastbook.setup_book()

from fastai.vision.all import *
from fastai.vision.widgets import *

import pandas as pd
import joblib
import numpy as np
import cv2

OSError: [WinError 193] %1 is not a valid Win32 application

## Pre-preparation

In [8]:
labels = pd.read_csv('./image.csv') # 讀取csv
labels['label'].value_counts() # 查看總標籤數量

T-Shirt       836
Shirt         448
Shorts        442
Pants         429
Dress         398
Hoodie        374
Outwear       367
Longsleeve    348
Skirt         288
Blazer        279
Undershirt    202
Name: label, dtype: int64

In [9]:
def get_x(r): return './images_original/' + r['label'] + "/" + r['name'] # create path to open images in the original folder
def get_y(r): return r['label'].split(' ') # split the labels using space as a delimitter

# Replace 'Not sure' with 'Not_sure'.
labels.loc[labels['label']=='Not sure','label'] = 'Not_sure'

# Add suffix '.jpg' to all image files in the dataframe to open them within the model.
labels['name'] = labels['name'] + '.jpg' 

# Keep only the images and their labels for the model to process.
label_df = labels[['name', 'label']]

# 去除重複職
label_df = label_df.dropna(axis=0, how='any')
label_df

,name,label
0,T-Shirt_1.jpg,T-Shirt
1,T-Shirt_2.jpg,T-Shirt
2,T-Shirt_3.jpg,T-Shirt
3,T-Shirt_4.jpg,T-Shirt
4,T-Shirt_5.jpg,T-Shirt
...,...,...
4406,Shorts_438.jpg,Shorts
4407,Shorts_439.jpg,Shorts
4408,Shorts_440.jpg,Shorts
4409,Shorts_441.jpg,Shorts


In [1]:
# Create DataBlock
dblock = DataBlock( blocks = (ImageBlock, MultiCategoryBlock),
                    get_x = get_x, get_y = get_y,
                    item_tfms = RandomResizedCrop(128, min_scale=0.35))  # ensure every item is of the same size

dls = dblock.dataloaders(label_df) # collates items from dataset into minibatches

dls.show_batch(nrows=3, ncols=3)

NameError: name 'DataBlock' is not defined

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context # 通過導入ssl模組改成不用驗證

learn = cnn_learner(dls, resnet18, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(5, base_lr=3e-3)

: 

: 

In [ ]:
# Get Predictions and target variables
preds,targs = learn.get_preds()

In [ ]:
learn.fit_one_cycle(1,1e-3)

In [ ]:
xs = torch.linspace(0.01,0.99,50)
accs = [accuracy_multi(preds, targs, thresh=i, sigmoid=False) for i in xs] # get_preds applies sigmoid activation function

plt.plot(xs,accs);

In [ ]:
learn.export(Path("./export.pkl"))
joblib.dump(learn,'joblib_export.pkl')

In [ ]:
clf2 = joblib.load('joblib_export.pkl')

In [ ]:
clf2.summary()  

In [ ]:
type(clf2)

In [ ]:
def ClassifierLoop(predict):
    label_list = predict[0]
    tensorBase = predict[2]
    
    tens_list = []
    for base in tensorBase:
        if base.item() > 0.5:
            tens_list.append(base.item())
        
    print(tens_list, label_list)
    print(label_list[np.argmax(tens_list)])
    
    return label_list[np.argmax(tens_list)]

In [ ]:
# 讀取圖檔
clf_ori = joblib.load('joblib_export.pkl')
# clf_aft = joblib.load('joblib_export_aft.pkl')

img = cv2.imread('./test_picture/shirt_24.jpg')
image = cv2.resize(img, (480, 600))

print(clf_ori.predict(img))
print("-------------------------------------")